In [ ]:
#!/bin/bash
set -e -x

reffn=/home/egenge01/projects/IGL_ref_mod/reference_ready/modified_reference_renamed.fasta
IG_loci=/home/egenge01/projects/12_sample_test/IG_loci.bed
masked_ref=/home/egenge01/projects/12_sample_test/reference_IGloci_masked.fasta
scratch=$PWD
mask_ref=${scratch}/ref_IG_masked.fasta

function get_read_support_vdj3 {
    while read sample asm_bam chr2_gene chr2_import chr22_gene chr22_import igh_gene igh_import ighc_gene ighc_import ccs_bam
    do
        base_outd="${scratch}/read_support/${sample}/imported_genes"
        bam_file="${scratch}/read_support/${sample}/ccs_to_pers/output.sorted.bam" # ccs reads to personalized reference
        ref="${scratch}/read_support/${sample}/ccs_to_pers/pers_ref.fasta" # personalized reference

        if [ ! -f "${bam_file}.bai" ]; then
            samtools index "$bam_file"
        fi

        for gene_type in "chr2" "chr22" "igh" #"ighc"
        do
            import_out="${base_outd}/${gene_type}/${sample}_make_gene_file_imported.csv"

            if [[ -f "$import_out" ]]; then
		modified_import_out="$import_out"
		
                tmp_file="${import_out}_read_support.tmp"
                echo "Total_Positions,Average_Coverage,Mismatched_Positions,Matched_Positions,Position_Mismatches,Position_Matches,Percent_Accuracy,Positions_With_At_Least_10x_Coverage,Fully_Spanning_Reads,Fully_Spanning_Reads_100%_Match" > "$tmp_file"
                header=$(head -n 1 "$import_out")
                IFS=',' read -ra header_cols <<< "$header"
                for i in "${!header_cols[@]}"; do
                    case "${header_cols[$i]}" in
                        "contig") contig_col=$i ;;
                        "REGION_start") start_col=$i ;;
                        "REGION_end") end_col=$i ;;
			"gene") gene_col=$i ;;  # Added case for 'gene'
                    esac
                done
		tmp_counts="${tmp_file}_counts"  # Temporary file to hold counts
		> "$tmp_counts"  # Clear or create the temp file for counts

                tail -n +2 "$import_out" | while IFS=, read -ra line
                do
                    contig="${line[$contig_col]}"
                    start=$(echo "${line[$start_col]}" | awk '{printf "%.0f", $1}')
                    end=$(echo "${line[$end_col]}" | awk '{printf "%.0f", $1}')
                    
		    gene="${line[$gene_col]}"  # Extract the 'gene' value using the identified column index
                    region="${contig}:${start}-${end}"

                    contig_filename=$(echo "$contig" | tr '/' '_')
                    tmp_bam="${base_outd}/${gene_type}/${contig_filename}_${start}_${end}.bam"
                    mkdir -p "$(dirname "$tmp_bam")"
                    samtools view -F 0x100 -F 0x800 -b "$bam_file" -o "$tmp_bam" -U "/dev/null" "${contig}:${start}-${end}"
                    samtools index "$tmp_bam"

		    samtools mpileup -f "$ref" -r "$region" "$tmp_bam" | \
			awk -v total_positions="$((end - start + 1))" -v sample="$sample" \
			'BEGIN {
    total_reads=0; mismatched_positions=0; matched_positions=0; positions_with_10x=0;
    mismatch_list=""; match_list="";
}
{
    total_reads += length($5);
    mismatches = length(gensub(/[.,]/, "", "g", $5));
    matches = length(gensub(/[^.,]/, "", "g", $5));
    mismatch_list = (mismatch_list == "" ? mismatches : mismatch_list ":" mismatches);
    match_list = (match_list == "" ? matches : match_list ":" matches);

    coverage = length($5);
    if (coverage >= 10) {
        positions_with_10x++;
    }

    mismatch_rate = mismatches / coverage;
    match_rate = matches / coverage;

    if (mismatch_rate > 0.2) {
        mismatched_positions++;
    }

    if (match_rate > 0.8) {
        matched_positions++;
    }
}
END {
    avg_reads_per_position = (total_positions > 0) ? total_reads / total_positions : 0;
    percent_accuracy = (matched_positions / total_positions) * 100;
    print total_positions, avg_reads_per_position, mismatched_positions, matched_positions, mismatch_list, match_list, percent_accuracy, positions_with_10x;}' OFS=',' >> "${tmp_file}_awk_out"
		    python match_subsequences3.py "$tmp_bam" "$contig" "$start" "$end" "$gene" "$import_out" > "${tmp_file}_py_out"
		    wait
		    paste -d ',' "${tmp_file}_awk_out" "${tmp_file}_py_out" >> "$tmp_file"
		    rm "${tmp_file}_awk_out" "${tmp_file}_py_out"
                    rm "$tmp_bam" "${tmp_bam}.bai"
                done

# New block to merge data and update Subject and Sample_Name
		if [[ -f "$import_out" && -f "$tmp_file" ]]; then
		    combined_file="${import_out%.csv}_combined.csv"
		    final_output="${import_out%.csv}_with_read_support.csv"
		    
		    # Merge the original import file with the new tmp file
		    paste -d ',' "$import_out" "$tmp_file" > "$combined_file"
		    rm -f "$tmp_file"
		    
		    # Update the Subject and Sample_Name columns in the final output
		    awk -v sample="$sample" 'BEGIN{FS=OFS=","} {
    if (NR == 1) {
        print;
    } else {
        $2 = sample;  # Update the 2nd column with $sample
        $3 = sample;  # Update the 3rd column with $sample
        print;
    }
}' "$combined_file" > "$final_output"

		    rm -f "$combined_file"
		fi
	    fi
        done
    done < extended_samples_paths.fofn
}

In [ ]:
#!/usr/bin/env python3

import csv
import sys
import pysam

def reverse_complement(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    return ''.join([complement[base] for base in seq[::-1]])

# Function to extract the correct sequence to compare against
def get_sequence_from_csv(import_csv, gene_key, contig, start, end):
    sequence = ""
    reverse_comp = False
    with open(import_csv, mode='r') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            csv_start = int(float(row['REGION_start']))
            csv_end = int(float(row['REGION_end']))
            if row['gene'] == gene_key and row['contig'] == contig and csv_start == start and csv_end == end:
                if 'sense' in row and row['sense'] == '-':
                    reverse_comp = True
                # Determine the correct column for the sequence based on the gene key
                if any(substring in gene_key for substring in ['IGKV', 'IGLV', 'IGHV']):
                    sequence_column = 'V-REGION'
                elif any(substring in gene_key for substring in ['IGKJ', 'IGLJ', 'IGHJ']):
                    sequence_column = 'J-REGION'
                elif 'IGHD' in gene_key:
                    sequence_column = 'D-REGION'
                elif any(substring in gene_key for substring in ['IGKC', 'IGLC']):
                    sequence_column = 'C-REGION'
                else:
                    continue  # Skip if no matching column is found
                sequence = row[sequence_column]
                if reverse_comp:
                    sequence = reverse_complement(sequence)
                break
    return sequence

# Function to count matching reads
def count_matching_reads(bamfile, chrom, start, end, sequence):
    full_span_count = 0
    perfect_match_count = 0
    samfile = pysam.AlignmentFile(bamfile, "rb")
    for read in samfile.fetch(chrom, start, end):
        if read.is_unmapped or read.is_secondary or read.is_supplementary:
            continue
        if read.reference_start <= start and read.reference_end >= end:
            full_span_count += 1
            read_seq = read.query_sequence
            if sequence in read_seq or reverse_complement(sequence) in read_seq:
                perfect_match_count += 1
    return full_span_count, perfect_match_count

# Main script execution
if __name__ == '__main__':
    bamfile = sys.argv[1]
    contig = sys.argv[2]
    start = int(sys.argv[3])
    end = int(sys.argv[4])
    gene_key = sys.argv[5]
    import_csv = sys.argv[6]  # The CSV file with the sequences

    sequence = get_sequence_from_csv(import_csv, gene_key, contig, start, end)
    if sequence:
        full_span_count, perfect_match_count = count_matching_reads(bamfile, contig, start, end, sequence)
        print(f"{full_span_count},{perfect_match_count}")
    else:
        print(f"No sequence found for gene {gene_key} in region {contig}:{start}-{end}", file=sys.stderr)

In [ ]:
function get_read_support_vdj3 {
    base_outd="${scratch}/read_support/${sample}/imported_genes"
    bam_file="${scratch}/read_support/${sample}/ccs_to_pers/output.sorted.bam"
    ref="${scratch}/read_support/${sample}/ccs_to_pers/pers_ref.fasta"

    # Create the base output directory if it doesn't exist
    mkdir -p "$base_outd"

    if [ ! -f "${bam_file}.bai" ]; then
        samtools index "$bam_file"
    fi

    for gene_type in "igh" "igk" "igl"
    do
        case "$gene_type" in
            "igh")
                import_out="$igh_digger"
                ;;
            "igk")
                import_out="$igk_digger"
                ;;
            "igl")
                import_out="$igl_digger"
                ;;
        esac
        mkdir -p "${base_outd}/${gene_type}"  # Create the directory for import_out

        if [[ -f "$import_out" ]]; then
            tmp_file="${scratch}/read_support/${sample}/tmp/$(basename "$import_out")_read_support.tmp"
            mkdir -p "$(dirname "$tmp_file")"
            echo "Total_Positions,Average_Coverage,Mismatched_Positions,Matched_Positions,Position_Mismatches,Position_Matches,Percent_Accuracy,Positions_With_At_Least_10x_Coverage,Fully_Spanning_Reads,Fully_Spanning_Reads_100%_Match" > "$tmp_file"
            
            header=$(head -n 1 "$import_out")
            IFS=',' read -ra header_cols <<< "$header"
            for i in "${!header_cols[@]}"; do
                case "${header_cols[$i]}" in
                    "contig") contig_col=$i ;;
                    "start") start_col=$i ;;
                    "end") end_col=$i ;;
                    "ASC_match") gene_col=$i ;;
                esac
            done

            tmp_counts="${tmp_file}_counts"
            > "$tmp_counts"

            tail -n +2 "$import_out" | while IFS=, read -ra line
            do
                contig="${line[$contig_col]}"
                start=$(echo "${line[$start_col]}" | awk '{printf "%.0f", $1}')
                end=$(echo "${line[$end_col]}" | awk '{printf "%.0f", $1}')
                
                gene="${line[$gene_col]}"
                region="${contig}:${start}-${end}"

                contig_filename=$(echo "$contig" | tr '/' '_')
                tmp_bam="${base_outd}/${gene_type}/${contig_filename}_${start}_${end}.bam"
                mkdir -p "$(dirname "$tmp_bam")"
                samtools view -F 0x100 -F 0x800 -b "$bam_file" -o "$tmp_bam" -U "/dev/null" "${contig}:${start}-${end}"
                samtools index "$tmp_bam"

                samtools mpileup -f "$ref" -r "$region" "$tmp_bam" | \
                awk -v total_positions="$((end - start + 1))" -v sample="$sample" \
                'BEGIN {
                    total_reads=0; mismatched_positions=0; matched_positions=0; positions_with_10x=0;
                    mismatch_list=""; match_list="";
                }
                {
                    total_reads += length($5);
                    mismatches = length(gensub(/[.,]/, "", "g", $5));
                    matches = length(gensub(/[^.,]/, "", "g", $5));
                    mismatch_list = (mismatch_list == "" ? mismatches : mismatch_list ":" mismatches);
                    match_list = (match_list == "" ? matches : match_list ":" matches);

                    coverage = length($5);
                    if (coverage >= 10) {
                        positions_with_10x++;
                    }

                    mismatch_rate = mismatches / coverage;
                    match_rate = matches / coverage;

                    if (mismatch_rate > 0.2) {
                        mismatched_positions++;
                    }

                    if (match_rate > 0.8) {
                        matched_positions++;
                    }
                }
                END {
                    avg_reads_per_position = (total_positions > 0) ? total_reads / total_positions : 0;
                    percent_accuracy = (matched_positions / total_positions) * 100;
                    print total_positions, avg_reads_per_position, mismatched_positions, matched_positions, mismatch_list, match_list, percent_accuracy, positions_with_10x;
                }' OFS=',' >> "${tmp_file}_awk_out"

                python /home/zmvanw01/git_repos/swrm_scripts/monkey/read-support/match_subsequences.py "$tmp_bam" "$contig" "$start" "$end" "$gene" "$import_out" > "${tmp_file}_py_out"
                echo "gene is $gene. awk_out is $(cat "${tmp_file}_awk_out"). py_out is $(cat "${tmp_file}_py_out")"
                paste -d ',' "${tmp_file}_awk_out" "${tmp_file}_py_out" >> "$tmp_file"
                rm "${tmp_file}_awk_out" "${tmp_file}_py_out"
                rm "$tmp_bam" "${tmp_bam}.bai"
            done

            if [[ -f "$import_out" && -f "$tmp_file" ]]; then
                combined_file="${scratch}/read_support/${sample}/output/${gene_type}/$(basename "$import_out" .csv)_combined.csv"
                final_output="${scratch}/read_support/${sample}/output/${gene_type}/$(basename "$import_out" .csv)_with_read_support.csv"
                mkdir -p "$(dirname "$combined_file")"
                python -c "import pandas as pd; df1 = pd.read_csv('$import_out'); df2 = pd.read_csv('$tmp_file'); combined = pd.concat([df1, df2], axis=1); combined.to_csv('$final_output', index=False)"
                #rm -f "$tmp_file"
                

               # awk -v sample="$sample" 'BEGIN{FS=OFS=","} {
                #    if (NR == 1) {
                 #       print;
                  #  } else {
                   #     $2 = sample;
                    #    $3 = sample;
                     #   print;
                  #  }
               # }' "$combined_file" > "$final_output"

               # rm -f "$combined_file"
            fi
        fi
    done
}

sample=$1
assemblies_fasta=$2
igh_digger=$3
igk_digger=$4
igl_digger=$5
ccs_bam=$6
monkey_mask_ref=/home/zmvanw01/ref/monkey/Child_17thApril2024_bothhap_IG_masked_unique.fasta
mkdir -p ${scratch}/read_support/${sample}

# Process digger option files using Python
for digger in "$igh_digger" "$igk_digger" "$igl_digger"
do
    new_file_path="${scratch}/read_support/${sample}/$(basename $digger)_modified.csv"
    python -c "import pandas as pd; df = pd.read_csv('$digger'); df['notes'] = df['notes'].str.replace(',', ';', regex=False) if 'notes' in df.columns else df; df.to_csv('$new_file_path', index=False)"
done

# Update the digger variables to point to the new files
igh_digger="${scratch}/read_support/${sample}/$(basename $igh_digger)_modified.csv"
igk_digger="${scratch}/read_support/${sample}/$(basename $igk_digger)_modified.csv"
igl_digger="${scratch}/read_support/${sample}/$(basename $igl_digger)_modified.csv"


get_read_support_vdj3

In [ ]:
#!/usr/bin/env python3 match_subsequences.py’

import csv
import sys
import pysam

def reverse_complement(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    return ''.join([complement[base] for base in seq[::-1]])

# Function to extract the correct sequence to compare against
def get_sequence_from_csv(import_csv, gene_key, contig, start, end):
    sequence = ""
    reverse_comp = False
    with open(import_csv, mode='r') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            csv_start = int(float(row['start']))
            csv_end = int(float(row['end']))
            #print(f"gene is:{gene_key}. contig is: {contig}. start is: {start}. end is {end}.")
            if row['ASC_match'] == gene_key and row['contig'] == contig and csv_start == start and csv_end == end:
                if 'sense' in row and row['sense'] == '-':
                    reverse_comp = True

                sequence = row['seq']
                if reverse_comp:
                    sequence = reverse_complement(sequence)
                break

    return sequence

# Function to count matching reads
def count_matching_reads(bamfile, chrom, start, end, sequence):
    full_span_count = 0
    perfect_match_count = 0
    samfile = pysam.AlignmentFile(bamfile, "rb")
    for read in samfile.fetch(chrom, start, end):
        if read.is_unmapped or read.is_secondary or read.is_supplementary:
            continue
        if read.reference_start <= start and read.reference_end >= end:
            full_span_count += 1
            read_seq = read.query_sequence
            if sequence in read_seq or reverse_complement(sequence) in read_seq:
                perfect_match_count += 1
    return full_span_count, perfect_match_count

# Main script execution
if __name__ == '__main__':
    bamfile = sys.argv[1]
    contig = sys.argv[2]
    start = int(sys.argv[3])
    end = int(sys.argv[4])
    gene_key = sys.argv[5]
    import_csv = sys.argv[6]  # The CSV file with the sequences

    sequence = get_sequence_from_csv(import_csv, gene_key, contig, start, end)
    if sequence:
        full_span_count, perfect_match_count = count_matching_reads(bamfile, contig, start, end, sequence)
        print(f"{full_span_count},{perfect_match_count}")
    else:
        print(f"No sequence found for gene {gene_key} in region {contig}:{start}-{end}", file=sys.stderr)

In [ ]:
#!/bin/bash
set -e -x

user=$(whoami)

input_file=$1
monkey_mask_ref=/home/zmvanw01/ref/monkey/Child_17thApril2024_bothhap_IG_masked_unique.fasta
cat $input_file | while read sample assemblies_fasta igh_digger igk_digger igl_digger ccs_bam monkey_mask_ref
do
    outdir=$PWD/monkey_processing/${sample}
    mkdir -p $outdir
    mkdir -p $PWD/logs
    sbatch --time=88:00:00 -p compute -o $PWD/logs/${sample}_job.txt --wrap="bash /home/zmvanw01/git_repos/swrm_scripts/monkey/read-support/get_read_support_VDJs_monkey.sh ${sample} ${assemblies_fasta} ${igh_digger} ${igk_digger} ${igl_digger} ${ccs_bam}"

    count=`squeue | grep $user | wc -l`
    while [ ${count} -gt 15 ]
    do
        sleep 1s
        count=`squeue | grep $user | wc -l`
    done
done